# Two Qubit CNOT Gate with Piccolo.jl

In [1]:
using Revise
using Piccolo
using SparseArrays
using LinearAlgebra

## System Hamiltonian

The Hamiltonian for this two qubit system example is given by

$$
H(u(t)) = g \hat a^\dagger \hat a \hat b^\dagger \hat b + a_1(t) (\hat a + \hat a^\dagger) + i a_2(t) (\hat a - \hat a^\dagger) + a_3(t) (\hat b + \hat b^\dagger) + i a_4(t)(\hat b - \hat b^\dagger) 
$$

where $\hat a$ and $\hat b$ are the annihilation operators for the first and second qubit respectively, $g$ is the coupling strength, and $u_i(t)$ are the control functions.


In [2]:
function TwoQubitSystem(levels::Int)

    g_coupling = 0.1 # GHz (linear units)

    # annihilation operator for qubit 1 
    â = lift(annihilate(levels), 1, 2)

    # annihilation operator for qubit 2
    b̂ = lift(annihilate(levels), 2, 2);

    # drift Hamiltonian
    H_drift = 2π * g_coupling * â' * â * b̂' * b̂

    # drive Hamiltonians
    H_drives = [
        2π * (â + â'),
        2π * 1.0im * (â - â'),
        2π * (b̂ + b̂'),
        2π * 1.0im * (b̂ - b̂') 
    ]

    return QuantumSystem(H_drift, H_drives);
end

TwoQubitSystem (generic function with 1 method)

In [3]:
levels_per_qubit = 2
system = TwoQubitSystem(levels_per_qubit);

## CNOT Unitary Gate

The goal gate that we will be optimizing for is the CNOT gate defined as

$$
U_{CNOT} = \begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 0 & 1 \\
0 & 0 & 1 & 0
\end{pmatrix}
$$


In [4]:
U_goal = [
    1 0 0 0;
    0 1 0 0;
    0 0 0 1;
    0 0 1 0
]; 

## Temporal Discretization Parameters

Here we will define the temporal discretization parameters for the control functions. We will use a total time of $T = 10$ and a time step of $\Delta t = 0.1$.

In [5]:
T = 15.0 # ns
N = 100
Δt = T / N;

## Pulse Constraints

Here we define bounds on the control as well as a constraint on the second derivative of the control that enforces smoothness of the pulse.

In [6]:
a_bound = 0.02 # GHz
dda_bound = 0.2; # bound on second derivative of control, to ensure smoothness

## Defining The Problem

Here we define the problem using the `UnitarySmoothPulseProblem` which handles setting up a smoothness constrained pulse optimization problem for a unitary gate.

In [7]:
# setting maximum number of iterations 
max_iter = 100 

prob = UnitarySmoothPulseProblem(
    system,
    U_goal,
    N,
    Δt;
    a_bound=a_bound,
    dda_bound=dda_bound,
    max_iter=max_iter
)

Here we plot the initial guess for the unitary and controls, which are stored as `Symbol`s, `:Ũ⃗` and `:a`, respectively. Note that the initial guess for the unitary is not random, it is in fact the geodesic path between the identity and the target unitary. This is not required for the optimizer to converge, but it can help speed up convergence.

In [ ]:
plot(prob.trajectory, [:Ũ⃗, :a]; ignored_labels=[:Ũ⃗])

Here we call the `solve!` method on the problem, which uses the interior point nonlinear solver IPOPT to optimize over the states and controls of the problem.

In [ ]:
QuantumCollocation.solve!(prob)

Let's now calculate the fidelity of the optimized unitary with the target unitary, via rollout with the full matrix exponential, this is achieved via the `unitary_fidelity` method.

In [ ]:
fid = unitary_fidelity(prob.trajectory, system)
println("Fidelity: $fid")

And now let's plot the final solution for the unitary and controls.

In [ ]:
plot(prob.trajectory, [:Ũ⃗, :a]; ignored_labels=[:Ũ⃗])

## Minimum Time CNOT

Now let's see how fast we can make this pulse, by setting up and solving a minimum time problem, using the solution we just found.

We will add an objective term to the problem that penalizes the total time, as well as constraint on the final state of the form

$$
\mathcal{F}(U(T)) \geq \mathcal{F}_{\text{target}}.
$$

This enforce that the final fidelity does not decrease below the threshold $\mathcal{F}_{\text{target}}$, while also allowing the phase to shift.

In [ ]:
# final fidelity constraint
min_fidelity = 0.9999

# minimum time objective weight
D = 1000.0

# define the problem
mintime_prob = UnitaryMinimumTimeProblem(prob; D=D, final_fidelity=min_fidelity);

In [ ]:
# solving the problem
QuantumCollocation.solve!(mintime_prob)

In [ ]:
mintime_final_fidelity = unitary_fidelity(mintime_prob.trajectory, system)
println("Fidelity: $mintime_final_fidelity")

In [ ]:
plot(mintime_prob.trajectory, [:Ũ⃗, :a]; ignored_labels=[:Ũ⃗])

In [ ]:
original_duration = times(prob.trajectory)[end]
mintime_duration = times(mintime_prob.trajectory)[end]
println("Original Duration: $original_duration ns")
println("Minimum Duration:  $mintime_duration ns")

## Model Mistmatch and Iterative Learning Control

Now let's imagine that our goal is to test this pulse on an experimental system where we know there are more levels than the two per qubit we used in our original model. We can simulate this by testing out pulse on a 3 level system.

In [ ]:
experimental_levels = 4
experimental_system = TwoQubitSystem(experimental_levels);

In [ ]:
# define quantum ground state |g⟩
g = cavity_state(0, experimental_levels)

# define quantum excited state |e⟩
e = cavity_state(1, experimental_levels)

# define quantum states |gg⟩, |ge⟩, |eg⟩, |ee⟩
gg = g ⊗ g
ge = g ⊗ e
eg = e ⊗ g
ee = e ⊗ e

# define CNOT gate
U = gg * gg' + ge * ge' + ee * eg' + eg * ee' 
U |> Matrix{Float64} |> sparse


In [ ]:
# define a measurement function that calculates the fidelity of the pulse evolving and initial |gg⟩ state to the target gate
measurement = ψ̃ -> [fidelity(iso_to_ket(ψ̃), U * gg)] 

# define the measurement times
τs = [prob.trajectory.T]

# define the simulation experiment
experiment = QuantumSimulationExperiment(
    experimental_system,
    ket_to_iso(gg),
    measurement,
    τs
);

In [ ]:
# rollout |gg⟩ with the pulse extract the final state

A = mintime_prob.trajectory.a
Δt = vec(mintime_prob.trajectory.Δt)

y = experiment(A, Δt).ys[end]

println("Fidelity: $y")

In [ ]:
y_goal = measurement(ket_to_iso(U * gg)) 
J(ŷ) = norm(ŷ - y_goal, 1)
optimizer = BacktrackingLineSearch(J);

In [ ]:
max_iter = 5

ILC_prob = ILCProblem(
    mintime_prob.trajectory, 
    mintime_prob.system, 
    mintime_prob.integrators,
    experiment, 
    optimizer;
    max_iter=max_iter
)

In [35]:
solve!(ILC_prob)


Iteration 1


    J(Aⁱ) = 0.015503231298470421



        α < cutoff



Iteration 2


    J(Aⁱ) = 0.015503231298470421



        α < cutoff



Iteration 3


    J(Aⁱ) = 0.015503231298470421



        α < cutoff



Iteration 4


    J(Aⁱ) = 0.015503231298470421



        α < cutoff



Iteration 5


    J(Aⁱ) = 0.015503231298470421



        α < cutoff
